In [5]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType


In [19]:
spark = SparkSession.builder.appName('pyspark').getOrCreate()

FILENAME = '../temperature_data.txt'
# regin,tik,uik,registered_voters,total_ballots,early_voters,tik_voters,home_voters,empty_ballots,home_ballots,tik_ballots,
# bad_ballots,normal_ballots,missing_ballots,extra_ballots,baburin,grudinin,zhirinovsky,putin,sobchak,suraykin,titov,yavlinsky
schema = StructType([
    StructField('Day', DateType(), False),
    StructField('t', IntegerType(), False),
])

df = spark.read.csv(FILENAME, schema=schema)



In [33]:
new_df = df.select(F.year("Day").alias("year"), F.month("Day").alias("month"), "t")

grouped = new_df.groupBy(['year','month']).agg(F.sum('t').alias('avg_t'))
grouped.sort(['year','month'], ascending=True).dropna()

grouped.groupBy('year').agg(F.stddev(F.col('avg_t'))).dropna().sort(F.col('stddev_samp(avg_t)'), ascending=False).show(1)


+----+------------------+
|year|stddev_samp(avg_t)|
+----+------------------+
|2010| 385.4535698958495|
+----+------------------+
only showing top 1 row



In [34]:
grouped.groupBy('year').agg(F.stddev(F.col('avg_t'))).dropna().sort(F.col('stddev_samp(avg_t)'), ascending=False).show()


+----+------------------+
|year|stddev_samp(avg_t)|
+----+------------------+
|2010| 385.4535698958495|
|2012|323.68344056650426|
|2011|  319.156404613883|
|2013|318.04102079620316|
|2016| 308.4830430976521|
|2014| 297.9674580564228|
|2009|  287.146304172629|
|2015| 244.4443916437041|
|2008|237.47438458354475|
+----+------------------+

